# **Deep Learning Exploration**

## **Load Data**

In [24]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [25]:
root_dir = "/content/drive/My Drive/"
personal_dir = 'Repo_Tesis/' 
base_dir = root_dir + personal_dir
print(base_dir)

/content/drive/My Drive/Repo_Tesis/


## **Data Augmentation**

En este apartado se genera la aumentación de datos, buscando que la aumentación sea aplicada a ambas clases se crean dos generadores con las mismas caracteristicas y posterior a eso se embeben en un iterable. Esta aumentación se le aplica a cada una de las imagenes y mascaras y se obtiene dos carpetas una con mascaras aumentadas y una con imagenes aumentadas, que posterior a esto se pueden usar para entrenar redes neuronales. 

In [31]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import os


# Define el nombre de la carpeta donde se guardarán las imágenes generadas
output_dir = os.path.join(base_dir, 'augmented_images')

# Crea las carpetas si aún no existen
if not os.path.exists(os.path.join(output_dir, 'mask')):
    os.makedirs(os.path.join(output_dir, 'mask'))
if not os.path.exists(os.path.join(output_dir, 'real_image')):
    os.makedirs(os.path.join(output_dir, 'real_image'))

# Define un generador de imágenes de entrenamiento para la imagen de máscara y para la imagen real
train_datagen_mask = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_datagen_real = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Carga las imágenes de máscara y de imágenes reales
train_generator_mask = train_datagen_mask.flow_from_directory(
    os.path.join(base_dir, 'Training/mask'),
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    shuffle=True,
    seed=42)

train_generator_real = train_datagen_real.flow_from_directory(
    os.path.join(base_dir, 'Training/real_img'),
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    shuffle=True,
    seed=42)

# Genera las imágenes aumentadas y guárdalas en las carpetas correspondientes
num_augmented_images = 10 # número de imágenes aumentadas por cada imagen original

for folder in os.listdir(os.path.join(base_dir, 'Training/mask')):
    if not os.path.exists(os.path.join(output_dir, 'mask', folder)):
        os.makedirs(os.path.join(output_dir, 'mask', folder))
    if not os.path.exists(os.path.join(output_dir, 'real_image', folder)):
        os.makedirs(os.path.join(output_dir, 'real_image', folder))
    
    for i in range(num_augmented_images):
        batch_mask, batch_real = next(zip(train_generator_mask, train_generator_real))
        augmented_images_mask = train_datagen_mask.flow(batch_mask[0], batch_mask[1]).next()[0]
        augmented_images_real = train_datagen_real.flow(batch_real[0], batch_real[1]).next()[0]

        for j in range(augmented_images_mask.shape[0]):
            Image.fromarray((augmented_images_mask[j]*255/np.max(augmented_images_mask[j])).astype('uint8')).save(os.path.join(output_dir, 'mask', folder, 'augmented_' + str(i) + str(j) + '.jpg'))
            Image.fromarray((augmented_images_real[j]*255/np.max(augmented_images_real[j])).astype('uint8')).save(os.path.join(output_dir, 'real_image', folder, 'augmented_' + str(i) + str(j) + '.jpg'))


Found 532 images belonging to 4 classes.
Found 532 images belonging to 4 classes.


## **Pruebas adicionales de data augmentation**

PD: No correr, esta en fase de prueba

In [26]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import os

# Define el nombre de la carpeta donde se guardarán las imágenes generadas
output_dir = os.path.join(base_dir, 'augmented_images')

# Crea las carpetas si aún no existen
if not os.path.exists(os.path.join(output_dir, 'mask')):
    os.makedirs(os.path.join(output_dir, 'mask'))
if not os.path.exists(os.path.join(output_dir, 'real_image')):
    os.makedirs(os.path.join(output_dir, 'real_image'))

# Define un generador de imágenes de entrenamiento para la imagen de máscara
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


# Carga las imágenes de máscara y de imágenes reales
train_generator = train_datagen.flow_from_directory(
       os.path.join(base_dir, 'Training'),
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical', 
        classes=['mask', 'real_ima'])

Found 532 images belonging to 2 classes.


In [13]:
# Define el nombre de la carpeta donde se guardarán las imágenes generadas
output_dir = os.path.join(base_dir, 'data_augmentation')

# Crea las carpetas si aún no existen
if not os.path.exists(os.path.join(output_dir, 'mask')):
    os.makedirs(os.path.join(output_dir, 'mask'))
if not os.path.exists(os.path.join(output_dir, 'real_image')):
    os.makedirs(os.path.join(output_dir, 'real_image'))

# Define un generador de imágenes de entrenamiento para ambas carpetas
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Carga las imágenes de máscara y de imágenes reales
train_mask_generator = train_datagen.flow_from_directory(
    os.path.join(base_dir, 'Training/mask'),
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical', 
    save_to_dir=os.path.join(output_dir, 'mask'),
    save_prefix='mask_')

train_real_generator = train_datagen.flow_from_directory(
    os.path.join(base_dir, 'Training/real_img'),
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical', 
    save_to_dir=os.path.join(output_dir, 'real_image'),
    save_prefix='real_')

# Combina los generadores de ambas carpetas en un solo generador
train_generator = zip(train_mask_generator, train_real_generator)


Found 532 images belonging to 4 classes.
Found 532 images belonging to 4 classes.


In [ ]:
# Generar y guardar imágenes con data augmentation
for i, (mask_batch, real_image_batch) in enumerate(train_generator):
    # Generar imágenes aumentadas
    mask_augmented_images = mask_datagen.flow(
        mask_batch[0], batch_size=32, save_to_dir=os.path.join(output_dir, 'mask'), save_prefix='aug_', save_format='png')
    real_image_augmented_images = real_image_datagen.flow(
        real_image_batch[0], batch_size=32, save_to_dir=os.path.join(output_dir, 'real_image'), save_prefix='aug_', save_format='png')

    # Guardar imágenes aumentadas
    for j in range(len(mask_batch[0])):
        mask_augmented_image = mask_augmented_images[j]
        real_image_augmented_image = real_image_augmented_images[j]
        mask_orig_image_path = mask_orig_images.filenames[i * 32 + j]
        real_image_orig_image_path = real_image_orig_images.filenames[i * 32 + j]
        mask_orig_image_name = os.path.basename(mask_orig_image_path)
        real_image_orig_image_name = os.path.basename(real_image_orig_image_path)
        mask_orig_image_dir = os.path.dirname(mask_orig_image_path)
        real_image_orig_image_dir = os.path.dirname(real_image_orig_image_path)

        mask_augmented_image_path = os.path.join(
            output_dir, 'mask', mask_orig_image_dir, 'aug_' + mask_orig_image_name)
        real_image_augmented_image_path = os.path.join(
            output_dir, 'real_image', real_image_orig_image_dir, 'aug_' + real_image_orig_image_name)

        mask_augmented_image.save(mask_augmented_image_path)
        real_image_augmented_image.save(real_image_augmented_image_path)

    # Salir del ciclo después de generar y guardar 100 imágenes aumentadas
    if i >= 3:
        break
